In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
import os

import numpy as np
import torch
from torch.nn import functional as F
import torch.cuda.amp as amp
import matplotlib.pyplot as plt
import IPython.display as ipd
import wandb

from deepspeech import model, train, datasets, utils, predict

## Prediction

In [ ]:
m, ptrain = predict.load('purzelrakete/feldberlin-deepspeech/212d0t74')
m.cfg.dataset

In [ ]:
ds = datasets.yesno(m.cfg)
ds = datasets.SpecAugmented(ds, m.cfg, masked=False)

In [ ]:
batch = [ds[i] for i in range(2)]
x, xn, y, yn = datasets.batch(m.cfg)(batch)